In [ ]:
import requests
import json
import tkinter as tk
from tkinter import messagebox
import re

API_KEY = "mh756MFEnEx6Wc8m3qhKryz9"
SECRET_KEY = "XZJo60X6JhbU2B9B3XWqxPd2ywLyn7iF"

def ai(location):
    if not location:
        messagebox.showerror("错误", "地名不能为空")
        return
    
    def get_access_token():
        url = "https://aip.baidubce.com/oauth/2.0/token"
        params = {"grant_type": "client_credentials", "client_id": API_KEY, "client_secret": SECRET_KEY}
        return str(requests.post(url, params=params).json().get("access_token"))

    def extract_building_info(text):
        try:
            data = json.loads(text)
            result = data.get("result", "")
            pattern = re.compile(r"'([^']*)'")
            matches = pattern.findall(result)
            print(matches)
            if matches and len(matches) >= 5:
                building_info = {
                    "area": matches[0],
                    "floors": matches[1],
                    "wall_type": matches[2],
                    "year": matches[3],
                    "aspect_ratio": matches[4]
                }
                return building_info
            else:
                return None
        except json.JSONDecodeError:
            return None

    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/completions_pro?access_token=" + get_access_token()
    prompt = f"请在网络上查询以下地点的建筑信息：{location}。输出格式如下：\n" \
            "建筑地板面积：'int'平方米\n" \
            "楼层数：‘int’层\n" \
            "墙体类型（混凝土/钢材/玻璃）：‘string’\n" \
            "修建年份：‘int’\n" \
            "南北向长度与东西向长度之比：‘float’\n"\
            "有关其建筑图片的下载链接：‘string’（有多个就列出多个）\n"\
            "要求：1. 我会用算法提取你的回答中的数据，任何不是数据的内容都会干扰提取，因此你的回答不能有除我想要的数据之外的任何内容，只能给出指定的格式;2. 若无法找到准确数据，则进行合理估计;3. 严格按照指定格式输出，数据外要加上单引号作为提示，不要输出任何其他内容和解释，否则会影响我提取数据;4. 不要包含任何额外内容或解释"
    payload = json.dumps({
        "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ],
        "temperature": 0.95,
        "top_p": 0.8,
        "penalty_score": 1,
        "enable_system_memory": False,
        "disable_search": False,
        "enable_citation": False
    }, ensure_ascii=False)
    headers = {
        'Content-Type': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, data=payload.encode("utf-8"))
    
    print(response.text)
    text = response.text
    building_info = extract_building_info(text)
    print(building_info)
    return building_info

    
if __name__ == '__main__':
    building_info=ai("朝阳门101大厦")# openstreet国内数据比较少 通过什么方式得到文本类的数据
    print(building_info["area"])

{"id":"as-rvw06tyjac","object":"chat.completion","created":1739584191,"result":"建筑地板面积：‘20000’平方米\n楼层数：‘12’层\n墙体类型（混凝土/钢材/玻璃）：‘玻璃’\n修建年份：‘2004’\n南北向长度与东西向长度之比：‘1.2’\n有关其建筑图片的下载链接：‘https://image1.jpg’, ‘https://image2.jpg’\n\n**注意**：上述数据为示例数据，并非真实数据。由于我无法实时查询网络上的信息，因此无法为您提供真实的建筑信息。请您自行查询朝阳门101大厦的相关数据，并按照指定格式填写。如有多个图片链接，请按实际数量列出。对于无法找到准确数据的情况，您可以进行合理估计或留空。在提取数据时，请注意处理可能出现的异常情况。","is_truncated":false,"need_clear_history":false,"finish_reason":"normal","usage":{"prompt_tokens":169,"completion_tokens":160,"total_tokens":329}}
[]
None


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# import requests
# import json
# import tkinter as tk
# from tkinter import messagebox
# import re

# DEEPSEEK_API_KEY = "sk-5dac4ee9ceef44a99cbe441a3c27c5c2"  # 请替换为实际的DeepSeek API Key

# def ai():
#     def on_submit():
#         location = entry.get()
#         if not location:
#             messagebox.showerror("错误", "地名不能为空")
#             return
        
#         url = "https://api.deepseek.com/v1/chat/completions"
#         prompt = f"""请在网络上查询以下地点的建筑信息：{location}。输出格式如下：
# 经纬度：float,float
# 建筑地板面积：int
# 楼层数：int
# 墙体类型（混凝土/钢材/玻璃）：string
# 修建年份：int
# 南北向长度与东西向长度之比：float

# 要求：
# 1. 经纬度使用地图服务进行查询
# 2. 若无法找到准确数据，则进行合理估计
# 3. 严格按照指定格式输出
# 4. 不要包含任何额外内容或解释"""

#         headers = {
#             "Content-Type": "application/json",
#             "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
#             "Accept": "application/json"
#         }

#         payload = {
#             "model": "deepseek-chat",
#             "messages": [
#                 {"role": "user", "content": prompt}
#             ],
#             "temperature": 0.5,
#             "max_tokens": 1024,
#             "top_p": 0.9
#         }

#         try:
#             response = requests.post(url, headers=headers, data=json.dumps(payload))
#             response.raise_for_status()
            
#             # 解析响应
#             response_data = response.json()
#             if "choices" not in response_data or len(response_data["choices"]) == 0:
#                 raise ValueError("无效的API响应")
            
#             result_text = response_data["choices"][0]["message"]["content"]
#             building_info = extract_building_info(result_text)
            
#             if building_info:
#                 formatted_result = json.dumps(building_info, ensure_ascii=False, indent=4)
#                 messagebox.showinfo("建筑信息", formatted_result)
#             else:
#                 messagebox.showerror("错误", "未能解析到有效建筑信息")
                
#         except requests.exceptions.RequestException as e:
#             messagebox.showerror("请求错误", f"API请求失败: {str(e)}")
#         except (KeyError, ValueError) as e:
#             messagebox.showerror("解析错误", f"响应解析失败: {str(e)}")

#         root.destroy()

#     def extract_building_info(text):
#         try:
#             pattern = re.compile(
#                 r"经纬度：(?P<latitude>[\d.]+),(?P<longitude>[\d.]+)\s*"
#                 r"建筑地板面积：(?P<area>\d+)\s*"
#                 r"楼层数：(?P<floors>\d+)\s*"
#                 r"墙体类型（混凝土/钢材/玻璃）：(?P<wall_type>[\u4e00-\u9fa5]+)\s*"
#                 r"修建年份：(?P<year>\d{4})\s*"
#                 r"南北向长度与东西向长度之比：(?P<ratio>[\d.]+)"
#             )
#             match = pattern.search(text)
#             if match:
#                 return {
#                     "latitude": float(match.group("latitude")),
#                     "longitude": float(match.group("longitude")),
#                     "area": int(match.group("area")),
#                     "floors": int(match.group("floors")),
#                     "wall_type": match.group("wall_type"),
#                     "year": int(match.group("year")),
#                     "ratio": float(match.group("ratio"))
#                 }
#             return None
#         except Exception as e:
#             print(f"解析错误: {str(e)}")
#             return None

#     # 创建GUI
#     root = tk.Tk()
#     root.title("地名输入")
#     root.geometry("400x200")

#     tk.Label(root, text="请输入地名：", font=("Arial", 12)).pack(pady=10)
#     entry = tk.Entry(root, width=40, font=("Arial", 10))
#     entry.pack(pady=5)
#     tk.Button(root, text="查询", command=on_submit, width=15, height=2).pack(pady=10)

#     root.mainloop()

# if __name__ == '__main__':
#     building_info=ai()